# Digital Filter Design with Python

This notebook introduces the basics of **digital filter design** using Python.  
We build directly on the concepts of poles, zeros, and transfer functions introduced in the *Z-transform fundamentals* notebook.

## 1. FIR Filters (Finite Impulse Response)

- Impulse response settles to zero in finite time.  
- Implemented only with **zeros** (numerator terms).  
- Always stable.  
- Can be designed with **linear phase**.

## 1.1 Key Filter Design Concepts

When designing FIR filters, you will see several important specifications:

- **Cutoff frequency ($f_c$)**:  
  The frequency at which the filter starts attenuating the signal.  
  For a **low-pass filter**, signals below $f_c$ are mostly passed, and signals above are reduced.  
  Cutoff is usually defined at the –3 dB point (≈ 70% of the passband voltage amplitude).

- **Transition band ($\Delta f$)**:  
  The frequency range between the passband and stopband.  
  A narrow transition band requires more filter coefficients (**more taps**).

- **Stopband attenuation ($A_{stop}$)**:  
  How much unwanted frequencies are suppressed.  
  Example: –60 dB means the stopband signal is reduced to 0.1% of its original voltage.

---

### Relating dB to ADC Voltage

Decibels (dB) are a **logarithmic scale**:

$$
\text{dB} = 20 \cdot \log_{10}\!\left(\frac{V}{V_{ref}}\right)
$$

where:
- $V$ is the measured signal amplitude,
- $V_{ref}$ is a reference level (often the **ADC full-scale range** or maximum measurable voltage).

**Example**:  
If an ADC has a 1 V reference:
- A signal at –20 dB means:  
  $V = V_{ref} \cdot 10^{-20/20} = 1 \cdot 0.1 = 0.1\ \text{V}$.
- A signal at –60 dB means:  
  $V = 1 \cdot 10^{-60/20} \approx 0.001\ \text{V}$.

So:  
- –20 dB = 10× smaller voltage than reference  
- –40 dB = 100× smaller  
- –60 dB = 1000× smaller  

---

### Practical Notes for Students

- Increasing **filter taps** makes the transition band narrower but requires more computation.  
- A higher **stopband attenuation** means better filtering but also needs more taps.  
- When looking at dB values in plots, you can always convert them to volts to check if they are significant compared to your ADC resolution.


In [5]:
from ipywidgets import interact, widgets
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt

def fir_design_demo(numtaps=51, cutoff=100, trans_width=50):
    fs = 1000
    # Define passband and stopband edges
    bands = [0, cutoff, cutoff+trans_width, fs/2]
    desired = [1, 0]  # passband gain =1, stopband gain=0
    
    # FIR design (remez / equiripple)
    b = signal.remez(numtaps, bands, desired, fs=fs)
    a = [1.0]
    
    # Frequency response
    w, h = signal.freqz(b, a, fs=fs)
    
    # Pole-zero plot
    z, p, k = signal.tf2zpk(b, a)
    
    plt.figure(figsize=(12,4))
    
    # Pole-zero plot
    ax1 = plt.subplot(1,2,1)
    ax1.scatter(np.real(z), np.imag(z), marker='o', label='Zeros')
    ax1.scatter(np.real(p), np.imag(p), marker='x', label='Poles')
    circle = plt.Circle((0,0), 1, color='black', fill=False, linestyle='--')
    ax1.add_artist(circle)
    ax1.axhline(0, color='gray'); ax1.axvline(0, color='gray')
    ax1.set_title("FIR Pole–Zero Plot"); ax1.set_xlabel("Re"); ax1.set_ylabel("Im")
    ax1.legend(); ax1.grid(); ax1.set_aspect('equal', adjustable='box')
    
    # Frequency response
    plt.subplot(1,2,2)
    plt.plot(w, 20*np.log10(np.abs(h)))
    plt.title("FIR Frequency Response")
    plt.xlabel("Frequency [Hz]"); plt.ylabel("Magnitude [dB]")
    plt.grid()
    
    plt.tight_layout()
    plt.show()

interact(
    fir_design_demo,
    numtaps=widgets.IntSlider(min=11, max=151, step=2, value=51, description="Taps"),
    cutoff=widgets.IntSlider(min=5, max=300, step=5, value=100, description="Cutoff [Hz]"),
    trans_width=widgets.IntSlider(min=10, max=200, step=10, value=50, description="Transition [Hz]")
)

interactive(children=(IntSlider(value=51, description='Taps', max=151, min=11, step=2), IntSlider(value=100, d…

<function __main__.fir_design_demo(numtaps=51, cutoff=100, trans_width=50)>

## 2. IIR Filters (Infinite Impulse Response)

- Impulse response theoretically lasts forever.  
- Implemented with **poles** (denominator terms).  
- More efficient for sharp filters, but may have nonlinear phase.  

## IIR Filter Design Parameters

When designing **IIR filters**, we often specify **order, cutoff frequency, and ripple/attenuation parameters**.  
These indirectly control the **transition band** and overall behavior of the filter.

### Key Parameters
- **Order**: Higher order = sharper cutoff, but more poles and potential stability issues.
- **Cutoff frequency ($f_c$)**: Where the filter begins attenuating signals. Defined relative to the sampling rate $f_s$.
- **Passband ripple ($R_p$)**: Allowed variation in the passband (in dB). Smaller $R_p$ = flatter passband.
- **Stopband attenuation ($R_s$)**: Minimum attenuation in the stopband (in dB). Higher $R_s$ = stronger suppression.

### Common IIR Filter Types
- **Butterworth**
  - Maximally flat passband (no ripple).
  - Smooth, monotonic roll-off.
  - Transition band is wider than Chebyshev/Elliptic for the same order.

- **Chebyshev Type I**
  - Allows ripple in the **passband** (controlled by $R_p$).
  - Steeper roll-off than Butterworth for same order.
  - Stopband is monotonic (no ripple).

- **Chebyshev Type II**
  - Flat passband (no ripple).
  - Ripple in the **stopband** (controlled by $R_s$).
  - Useful when passband must remain very flat.

- **Elliptic**
  - Ripple in both passband and stopband ($R_p$ and $R_s$).
  - Provides the **sharpest transition** for a given order.
  - Most efficient in terms of order, but least "clean" visually.

### Practical Notes
- If you want **smooth passband**, choose **Butterworth**.  
- If you want **steeper cutoff and can accept passband ripple**, choose **Chebyshev I**.  
- If you need a **very flat passband but don’t care about stopband ripple**, use **Chebyshev II**.  
- If you need the **sharpest transition band** with lowest order, choose **Elliptic**.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import ipywidgets as widgets
from ipywidgets import interact

fs = 1000  # Sampling frequency

def iir_demo(order=4, cutoff=100, ftype='butter', rp=1, rs=40):
    # Design IIR filter depending on type
    if ftype == 'butter':
        b, a = signal.butter(order, cutoff, fs=fs)
        desc = "Butterworth (flat passband)"
    elif ftype == 'cheby1':
        b, a = signal.cheby1(order, rp, cutoff, fs=fs)
        desc = f"Chebyshev I (Rp={rp} dB)"
    elif ftype == 'cheby2':
        b, a = signal.cheby2(order, rs, cutoff, fs=fs)
        desc = f"Chebyshev II (Rs={rs} dB)"
    elif ftype == 'ellip':
        b, a = signal.ellip(order, rp, rs, cutoff, fs=fs)
        desc = f"Elliptic (Rp={rp} dB, Rs={rs} dB)"
    else:
        raise ValueError("Unknown filter type")
    
    # Frequency response
    w, h = signal.freqz(b, a, fs=fs)
    z, p, _ = signal.tf2zpk(b, a)
    
    # Plot
    plt.figure(figsize=(12,5))
    
    # Pole–zero plot
    ax1 = plt.subplot(1,2,1)
    ax1.scatter(np.real(z), np.imag(z), marker='o', label='Zeros')
    ax1.scatter(np.real(p), np.imag(p), marker='x', label='Poles')
    circle = plt.Circle((0,0), 1, color='black', fill=False, linestyle='--')
    ax1.add_artist(circle)
    ax1.axhline(0, color='gray'); ax1.axvline(0, color='gray')
    ax1.set_title(f"{desc}\nPole–Zero Plot")
    ax1.set_xlabel("Re"); ax1.set_ylabel("Im")
    ax1.legend(); ax1.grid()
    
    # Keep aspect ratio equal, unit circle always visible
    ax1.set_aspect('equal', adjustable='box')
    margin = 0.2
    all_points = np.concatenate([z, p, [1, -1, 1j, -1j]])
    x_min, x_max = np.min(np.real(all_points)), np.max(np.real(all_points))
    y_min, y_max = np.min(np.imag(all_points)), np.max(np.imag(all_points))
    span = max(x_max - x_min, y_max - y_min) / 2
    center_x = (x_max + x_min) / 2
    center_y = (y_max + y_min) / 2
    ax1.set_xlim(center_x - span - margin, center_x + span + margin)
    ax1.set_ylim(center_y - span - margin, center_y + span + margin)
    
    # Magnitude and phase response
    ax2 = plt.subplot(1,2,2)
    ax2.plot(w, 20*np.log10(np.abs(h)), label="Magnitude [dB]")
    #ax2.plot(w, np.angle(h), label="Phase [rad]")
    ax2.set_title(f"{desc}\nFrequency Response")
    ax2.set_xlabel("Frequency [Hz]"); ax2.set_ylabel("Magnitude")
    ax2.grid(); ax2.legend()
    
    plt.tight_layout()
    plt.show()

# Interactive widget
interact(
    iir_demo,
    order=widgets.IntSlider(min=2, max=10, step=1, value=4, description="Order"),
    cutoff=widgets.IntSlider(min=50, max=fs//2-10, step=10, value=100, description="Cutoff [Hz]"),
    ftype=widgets.Dropdown(options=['butter','cheby1','cheby2','ellip'], value='butter', description="Type"),
    rp=widgets.FloatSlider(min=0.1, max=5, step=0.1, value=1, description="Rp (dB)"),
    rs=widgets.IntSlider(min=20, max=100, step=5, value=40, description="Rs (dB)")
)

interactive(children=(IntSlider(value=4, description='Order', max=10, min=2), IntSlider(value=100, description…

<function __main__.iir_demo(order=4, cutoff=100, ftype='butter', rp=1, rs=40)>

## 3. Practical Considerations: Implementation on ESP32

When designing filters for embedded systems, it is important to think about **computational cost** and **execution time**, not just frequency response.

### 3.1 FIR Filters
- Each output sample requires **N multiplications and N additions**,  
  where **N = number of filter taps**.
- Example: a 200-tap FIR filter → **200 multiplies + 200 adds per sample**.
- If the sampling rate is **200 Hz**, that means:
  - **40,000 operations per second**.
- On an ESP32 (≈ 200–300 MIPS depending on core and clock), this is feasible,  
  but wastes CPU cycles if simpler filters achieve the same goal.

### 3.2 IIR Filters
- Implemented as cascaded **biquads** (second-order sections).
- Each biquad requires about **5 multiplications + 5 additions per sample**.
- Example: a 6th-order IIR filter = 3 biquads → **15 multiplies per sample**.
- Much more efficient than a long FIR for steep attenuation.

---

### 3.3 Approximate CPU Load on ESP32 (200 Hz sampling)

| Filter type       | Complexity per sample | Operations/s | Approx. CPU load (240 MHz) |
|-------------------|-----------------------|--------------|-----------------------------|
| FIR, 50 taps      | 100 ops               | 20,000       | ~0.01%                      |
| FIR, 200 taps     | 400 ops               | 80,000       | ~0.03%                      |
| FIR, 500 taps     | 1000 ops              | 200,000      | ~0.08%                      |
| IIR, 6th order    | ~30 ops               | 6,000        | ~0.003%                     |

*Assumptions:*  
- One multiplication or addition ≈ one instruction cycle.  
- ESP32 at 240 MHz (240 million cycles/s).  
- Other system overhead not included.

---

### 3.4 Summary
- **FIR** filters scale linearly with taps. Long filters may become expensive.  
- **IIR** filters remain efficient even at higher orders.  
- Always check:  
  - **operations/sample × sampling rate**  
  - compare with available CPU cycles.  
- In practice:  
  - Use **FIR** when you need *linear phase*.  
  - Use **IIR** when efficiency and sharp roll-off are more important.


## 4. Exercise: Filtering Hand Movement Data

Imagine you are building a **gesture recognition system** that uses an accelerometer or IMU to capture **human hand movements**.  
- Typical **gesture frequencies** are in the range **0–10 Hz**.  
- The environment has **strong mains interference** at **50 Hz or 60 Hz** (depending on the country).  
- Your system must **suppress the interference by at least 80 dB** to avoid false detections.

### Tasks
1. **Filter type choice**  
   - Should you use an **FIR** or an **IIR** filter for this task?  
   - Discuss trade-offs between sharp cutoff, computation load, and phase linearity.

2. **Design the filter**  
   - Design a **low-pass filter** with passband up to 10 Hz.  
   - Ensure the **stopband starts at 50 Hz (or 60 Hz)**.  
   - Require **stopband attenuation ≥ 80 dB**.  
   - Sampling frequency: assume $f_s = 200$ Hz (you can adjust if needed).

3. **Plot results**  
   - Show the **magnitude response**.  
   - Verify that the stopband meets the **80 dB attenuation** requirement.  

4. **Discussion**  
   - What happens to the **transition band** if you make the filter FIR vs IIR?  
   - How many FIR taps would be needed to meet the 80 dB attenuation?  
   - Why might IIR be more practical in an embedded system for this application?



### 1. Filter type choice  

I would use an **IIR low-pass filter** for this task.  

It can reach the needed **−80 dB suppression** at **50/60 Hz** with **fewer steps**,so it requires **less computation** and results in a **lower CPU load**.  

A **FIR filter** could also achieve this, but it would need **many taps** and therefore run **much slower**.  

The **IIR filter** provides a **sharper cutoff**, but its **phase is not linear** - that’s fine because small phase shifts do not affect **hand-gesture data**.  

**Summary:**  
> **IIR** is smaller and faster, while **FIR** is heavier but has linear phase.

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import ipywidgets as widgets
from ipywidgets import interact

def make_filter_designer():
    """
    Self-contained designer: no globals leaked.
    Does not interfere with other cells (e.g., teacher's demo).
    """

    # --- Local spec (task) ---
    _fs  = 200.0   # Sampling frequency [Hz]
    _fp  = 10.0    # Passband edge [Hz]
    _ws  = 50.0    # Stopband start [Hz]
    _Rp  = 1.0     # Passband ripple [dB]
    _Rs  = 80.0    # Stopband attenuation [dB]

    def _design_to_spec(ftype='ellip', order_val=0):
        """
        Local design function (names are prefixed/hidden).
        order_val = 0 -> auto-pick minimal order; otherwise manual order.
        """

        # --- 1) Design filter (auto vs manual) ---
        if order_val == 0:  # AUTO
            if ftype == 'butter':
                N, Wn = signal.buttord(_fp, _ws, _Rp, _Rs, fs=_fs)
                b, a  = signal.butter(N, Wn, btype='low', fs=_fs)
                desc  = 'Butterworth'
            elif ftype == 'cheby1':
                N, Wn = signal.cheb1ord(_fp, _ws, _Rp, _Rs, fs=_fs)
                b, a  = signal.cheby1(N, _Rp, Wn, btype='low', fs=_fs)
                desc  = f'Chebyshev I (Rp={_Rp} dB)'
            elif ftype == 'cheby2':
                N, Wn = signal.cheb2ord(_fp, _ws, _Rp, _Rs, fs=_fs)
                b, a  = signal.cheby2(N, _Rs, Wn, btype='low', fs=_fs)
                desc  = f'Chebyshev II (Rs={_Rs} dB)'
            elif ftype == 'ellip':
                N, Wn = signal.ellipord(_fp, _ws, _Rp, _Rs, fs=_fs)
                b, a  = signal.ellip(N, _Rp, _Rs, Wn, btype='low', fs=_fs)
                desc  = f'Elliptic (Rp={_Rp} dB, Rs={_Rs} dB)'
            mode_text = "(auto)"
        else:             # MANUAL
            N = int(order_val)
            if ftype == 'butter':
                b, a = signal.butter(N, _fp, btype='low', fs=_fs)
                desc = 'Butterworth'
            elif ftype == 'cheby1':
                b, a = signal.cheby1(N, _Rp, _fp, btype='low', fs=_fs)
                desc = f'Chebyshev I (Rp={_Rp} dB)'
            elif ftype == 'cheby2':
                b, a = signal.cheby2(N, _Rs, _ws, btype='low', fs=_fs)
                desc = f'Chebyshev II (Rs={_Rs} dB)'
            elif ftype == 'ellip':
                b, a = signal.ellip(N, _Rp, _Rs, _fp, btype='low', fs=_fs)
                desc = f'Elliptic (Rp={_Rp} dB, Rs={_Rs} dB)'
            mode_text = "(manual)"

        # --- 2) Frequency response + verify -80 dB at 50 Hz ---
        w, h   = signal.freqz(b, a, fs=_fs)
        mag_db = 20*np.log10(np.maximum(np.abs(h), 1e-12))
        att_db = float(np.interp(_ws, w, mag_db))
        status_ok = (att_db <= -80)

        # --- 3) Plot ---
        plt.figure(figsize=(8,5))
        plt.plot(w, mag_db, label=f'{desc}, order={N} {mode_text}')
        plt.axvline(_fp, ls='--', color='red',   lw=1.2, label=f'cutoff {_fp} Hz')
        plt.axvline(_ws, ls='--', color='black', lw=1.0, label='mains 50 Hz')
        plt.axhline(-80, ls=':', color='gray',   lw=1.2, label='-80 dB target')
        plt.xlim(0, _fs/2); plt.ylim(-140, 5)
        plt.xlabel('Frequency [Hz]'); plt.ylabel('Magnitude [dB]')
        plt.title('Low-pass design (fs=200 Hz, stopband from 50 Hz)')
        plt.grid(True, linestyle=':')
        plt.legend()
        plt.show()

        # --- 4) Report (colored PASS/FAIL) ---
        GREEN = "\033[92m"; RED = "\033[91m"; RESET = "\033[0m"
        status_text = f"{GREEN}PASS{RESET}" if status_ok else f"{RED}FAIL{RESET}"
        print(f"Type: {desc}")
        print(f"Order used: {N} {mode_text}")
        print(f"Attenuation at 50 Hz: {att_db:.1f} dB  -> {status_text}")

    # --- Widgets (aligned) ---
    common_style  = {'description_width': '120px'}
    common_layout = widgets.Layout(width='420px')

    _order_slider = widgets.IntSlider(
        min=0, max=10, step=1, value=0,
        description='Order (0 = auto)',
        style=common_style, layout=common_layout
    )
    _type_dropdown = widgets.Dropdown(
        options=['ellip', 'cheby1', 'cheby2', 'butter'],
        value='ellip',
        description='Type',
        style=common_style, layout=common_layout
    )

    return interact(
        _design_to_spec,
        ftype=_type_dropdown,
        order_val=_order_slider
    )

make_filter_designer()

interactive(children=(Dropdown(description='Type', layout=Layout(width='420px'), options=('ellip', 'cheby1', '…

<function __main__.make_filter_designer.<locals>._design_to_spec(ftype='ellip', order_val=0)>

### 4. Discussion

#### Transition Band and FIR Taps

The **transition band** is the area where the filter changes from keeping useful signals (≤10 Hz) to blocking noise (≥50 Hz).

- In the **FIR filter**, this band is **wide and smooth** by default. To make it **sharper (narrower)**, we must increase the **number of taps**. The sharper we want the cutoff, the more taps we need.
- In the **IIR filter**, the transition band is already **steep and narrow**, even with a low filter order.

From the FIR example below (Kaiser window design):  
- For our spec (**10–50 Hz**, **−80 dB stopband**), the filter needs about **27 taps**.  
- If we decrease the **transition width**, the required number of taps grows quickly.

**In short:**  
FIR → smooth and wide transition, more taps needed for sharper cutoff.  
IIR → sharp transition with fewer coefficients.

#### Why IIR is More Practical

An **IIR filter** is more practical for this embedded system because it is **much lighter and faster** than an FIR filter.

- It needs only a **few coefficients (order 4–6)** instead of dozens or hundreds of taps.  
- This means **less computation** and **lower memory use**.  
- The IIR filter also gives a **sharp cutoff** and easily reaches **−80 dB** attenuation.  
- The small phase shift it causes does **not affect hand-gesture data**.

**Summary:**  
IIR is **smaller, faster, and efficient** - perfect for real-time filtering on embedded devices.

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import ipywidgets as widgets
from ipywidgets import interact

def make_fir_transition_demo():
    """
    Self-contained FIR (Kaiser) demo.
    - Does not interfere with other code (local variables only)
    - Safe to run alongside the teacher’s examples
    - Shows how Transition [Hz] affects the number of taps and attenuation
    """

    _fs = 200.0   # Sampling rate [Hz]
    _fp = 10.0    # Passband edge [Hz]
    _Rs = 80.0    # Fixed stopband attenuation target [dB]

    def _fir_kaiser_with_transition(transition=40.0):
        """
        FIR low-pass filter using a Kaiser window.
        The stopband starts at fsb = fp + transition.
        Rs is fixed to 80 dB (from the assignment).
        """
        fsb = _fp + transition                       # Stopband start [Hz]
        fsb = min(fsb, _fs/2 - 1e-6)                 # Keep below Nyquist limit

        # Normalized transition width (0...1 range relative to Nyquist)
        width_norm = (fsb - _fp) / (_fs/2)
        width_norm = max(min(width_norm, 0.999), 1e-4)  # Safety clamp

        # 1) Estimate number of taps and Kaiser beta to meet Rs=80 dB
        numtaps, beta = signal.kaiserord(ripple=_Rs, width=width_norm)

        # 2) Design FIR low-pass filter
        b = signal.firwin(numtaps=numtaps, cutoff=_fp, window=('kaiser', beta), fs=_fs)

        # 3) Compute frequency response and measure attenuation at fsb
        w, h = signal.freqz(b, [1.0], fs=_fs)
        mag_db = 20*np.log10(np.maximum(np.abs(h), 1e-12))
        att_fsb = float(np.interp(fsb, w, mag_db))
        status = "PASS" if att_fsb <= -80 else "FAIL"

        # 4) Plot the magnitude response
        plt.figure(figsize=(8,5))
        plt.plot(w, mag_db, label=f'FIR (Kaiser), taps={numtaps}')
        plt.axvline(_fp,  ls='--', color='red',   lw=1.2, label=f'passband ≤ {_fp} Hz')
        plt.axvline(fsb,  ls='--', color='black', lw=1.0, label=f'stopband from {fsb:.0f} Hz')
        plt.axhline(-80,  ls=':',  color='gray',  lw=1.0, label='-80 dB target')
        plt.xlim(0, _fs/2); plt.ylim(-140, 5)
        plt.xlabel('Frequency [Hz]'); plt.ylabel('Magnitude [dB]')
        plt.title('FIR (Kaiser) — Transition width vs taps (Rs fixed at 80 dB)')
        plt.grid(True, linestyle=':')
        plt.legend()
        plt.show()

        # 5) Print a short report
        print(f"fs={_fs:.0f} Hz | passband≤{_fp:.0f} Hz | stopband from {fsb:.0f} Hz | Transition={transition:.0f} Hz | Rs={_Rs:.0f} dB (fixed)")
        print(f"Normalized transition width: {width_norm:.3f}")
        print(f"Estimated taps: {numtaps}, beta={beta:.2f}")
        print(f"Attenuation @ {fsb:.0f} Hz: {att_fsb:.1f} dB -> {status}")

    # Widget: Transition [Hz] only
    _transition_slider = widgets.FloatSlider(
        min=5.0, max=80.0, step=1.0, value=40.0, description='Transition [Hz]'
    )

    return interact(
        _fir_kaiser_with_transition,
        transition=_transition_slider
    )

make_fir_transition_demo()

interactive(children=(FloatSlider(value=40.0, description='Transition [Hz]', max=80.0, min=5.0, step=1.0), Out…

<function __main__.make_fir_transition_demo.<locals>._fir_kaiser_with_transition(transition=40.0)>